This notebook allows to load the posterior used to compute summary statistics of galaxy surveys (in 3D_haloes.ipynb)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from random import *

from scipy.spatial.transform import Rotation

import torch
from torch.distributions import Uniform

from sbi.utils import BoxUniform
from sbi.analysis import pairplot
from sbi.inference import NPE, SNPE, simulate_for_sbi, infer
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
    prepare_for_sbi
)

import corner

from astropy.io import fits

/opt/homebrew/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Downloading the Abacus haloes (z=0.725)

In [7]:
nb_haloes=25_000


haloes_table1 = Table.read('haloes_abacus.fits')
haloes_table1 = haloes_table1[:nb_haloes]

### Some functions required for further work...

- ...to compute ellipticities

In [11]:
def get_galaxy_orientation_angle(e1,e2):
    return 0.5*np.arctan2(e2,e1)

def e_complex(a,b,r):
    abs_e = (1-(b/a)) / (1+(b/a))
    e1 = abs_e*np.cos(2*r)
    e2 = abs_e*np.sin(2*r)
    return e1, e2

def abs_e(e1,e2):
    return np.sqrt(e1*e1+e2*e2)

def a_b(e1,e2):
    e = abs_e(e1,e2)
    return 1+e,1-e 

- ...to format ellipsoidal haloes to match Abacus

In [12]:
def format_ellipsoid(eigenvectors, eigenvalues, position = np.asarray([0,0,0])):
    '''
    Formatt ellipsoid parameters to match Abacus. 
    Eigenvectors and values must be in order of least to greatest
    '''
    el = Table()
    el['sigman_eigenvecsMin_L2com'] = eigenvectors[0]
    el['sigman_eigenvecsMid_L2com'] = eigenvectors[1]
    el['sigman_eigenvecsMaj_L2com'] = eigenvectors[2]
    
    el['sigman_L2com'] = np.sqrt(eigenvalues)
    el['sigma_L2com'] = position
    
    return el

### Model : 3D galaxies populating haloes

- Parameters : $\theta = \{\mu_{\tau_B}, \mu_{\tau_C}, \sigma_{\tau_B}, \sigma_{\tau_C}, r_\tau\}$

In [14]:
def population_3D (tau_B, tau_C, sigma_tau_B, sigma_tau_C, r_tau, el=haloes_table1, nb_haloes=nb_haloes) : 

    haloes_table2 = el.copy()
    eigenvalues_orig = np.array(haloes_table2['sigman_L2com'])

    valid_eigenvalues = []


    while len(valid_eigenvalues) < nb_haloes:
    
        taus = np.random.multivariate_normal(mean=[tau_B, tau_C], cov=[[sigma_tau_B**2, r_tau * sigma_tau_B * sigma_tau_C],[r_tau * sigma_tau_B * sigma_tau_C, sigma_tau_C**2]],size=nb_haloes)
    
        tau_B2 = np.clip(taus[:, 0], 0, 1)
        tau_C2 = np.clip(taus[:, 1], 0, 1)

        eigenvalues = eigenvalues_orig.copy()
        eigenvalues[:, 1] *= tau_B2
        eigenvalues[:, 2] *= tau_C2

        Ag, Bg, Cg = eigenvalues[:, 0], eigenvalues[:, 1], eigenvalues[:, 2]
        mask = (Bg / Ag <= 1) & (Cg / Ag <= 1) & (Bg >= Cg) & (Bg > 0) & (Cg > 0)
        filtered = eigenvalues[mask]

        to_add = min(nb_haloes - len(valid_eigenvalues), filtered.shape[0])
        valid_eigenvalues.extend(filtered[:to_add])


    eigenvalues = np.array(valid_eigenvalues) 
   
    
    eigenvecs_Min = haloes_table2['sigman_eigenvecsMin_L2com'][:len(eigenvalues)]
    eigenvecs_Mid = haloes_table2['sigman_eigenvecsMid_L2com'][:len(eigenvalues)]
    eigenvecs_Max = haloes_table2['sigman_eigenvecsMaj_L2com'][:len(eigenvalues)]

    eigenvectors = np.stack((eigenvecs_Min, eigenvecs_Mid, eigenvecs_Max), axis=1)

    ellipses = np.array([format_ellipsoid(eigenvectors[i, :, :], eigenvalues[i, :]) for i in range(nb_haloes)])
    ellipses = np.array(ellipses)

    evcl = np.array([ellipses['sigman_eigenvecsMaj_L2com'], ellipses['sigman_eigenvecsMid_L2com'], ellipses['sigman_eigenvecsMin_L2com']])
    evcl=np.transpose(evcl, (1, 0, 2))
    evls = ellipses['sigman_L2com']**2 

    
    return evcl, evls

### Simulator to project 3D galaxies and their host-haloes in 2D

In [15]:
# SIMULATOR (for simulation-based inference): projection of the 3D galaxies-halos in 2D along the line-of-sight ('y')
# Output = summary statistics = P(e)


def simulator(theta, 
                el=haloes_table1,
                nb_haloes=nb_haloes, 
                p_axis='y', # The direction of projection (here 'y' denotes by convention the direction of the line-of-sight)
                e_bins=np.linspace(0,1,100) # The number of bins for the histogram of e_counts (output)
               ):

    tau_B, tau_C, sigma_tau_B, sigma_tau_C, r_tau = theta

    evcl, evls = population_3D (tau_B, tau_C, sigma_tau_B, sigma_tau_C, r_tau, el, nb_haloes)
    

    # Projection 3D => 2D
    if p_axis=='x': # Projection perpendicular to the LOS
        K = np.sum(evcl[:,:,0][:,:,None]*(evcl/evls[:,None]), axis=1)
        r = evcl[:,:,2] - evcl[:,:,0] * K[:,2][:,None] / K[:,0][:,None]
        s = evcl[:,:,1] - evcl[:,:,0] * K[:,1][:,None] / K[:,0][:,None] 

    if p_axis=='y': # Projection along the LOS
        K = np.sum(evcl[:,:,1][:,:,None] * (evcl/evls[:,None]), axis=1)
        r = evcl[:,:,0] - evcl[:,:,1] * K[:,0][:,None] / K[:,1][:,None]
        s = evcl[:,:,2] - evcl[:,:,1] * K[:,2][:,None] / K[:,1][:,None]


    # Coefficients A,B,C (eq 23 of (2))
    A1 = np.sum(r**2 / evls, axis=1)
    B1 = np.sum(2*r*s / evls, axis=1)
    C1 = np.sum(s**2 / evls, axis=1)


    # Axis a_p,b_p and orientation angle r_p of the projected galaxy
    r_p = np.pi / 2 + np.arctan2(B1,A1-C1)/2
    a_p = 1/np.sqrt((A1+C1)/2 + (A1-C1)/(2*np.cos(2*r_p)))
    b_p = 1/np.sqrt(A1+C1-(1/a_p**2))


    # Projected ellipticity
    e1, e2 = e_complex(a_p, b_p, r_p) ; e = [e1,e2] ; e=np.array(e)


    # Final output = summary statistics = P(e)
    e_counts,_ = np.histogram(np.sqrt(e[0,:]**2+e[1,:]**2),bins=e_bins)
    

    return e_counts/nb_haloes

### Priors for the parameters

In [17]:
class CustomPrior(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Define uniform priors for the parameters
        self.prior_uniform = BoxUniform(
            low=torch.tensor( [0, 0.0, 0, 0, -0.8]), 
            high=torch.tensor([1, 1.0, 1, 1, 0.8])
        )
    
    def log_prob(self, x):
        # Check the constraint mu_tau_B > mu_tau_C
        mutauB_minus_mutauC = x[..., 1] - x[..., 0]  # assuming mu_tauB is at index 0 and mu_C at index 1
        mask = ( mutauB_minus_mutauC > 0 )
        return torch.where(mask, self.prior_uniform.log_prob(x), torch.tensor(-float('inf')))

    
    def sample(self, sample_shape=torch.Size()):
        if len(sample_shape) == 0:
            sample_shape = torch.Size([1])  # Default to a single sample if sample_shape is empty
            
        samples = []
        while len(samples) < sample_shape[0]:
            sample = self.prior_uniform.sample((1,))
            if ( -sample[..., 0] + sample[..., 1]).lt(0):  # this checks if it is less than zero, so I switched it around
                samples.append(sample)
                
        return torch.cat(samples, dim=0)

# Instantiate the custom prior
prior = CustomPrior()

### Running the simulations

In [ ]:
# Running the simulations, training of the neural network and estimation of the posterior (NPE = Neural Posterior Estimation) 

posterior = infer(simulator, prior, method = 'NPE', num_simulations = 60000, num_workers = 30)

### Saving the estimated posterior

In [ ]:
torch.save(posterior,'posterior_abacus.pt')